In [ ]:
!apt install git ffmpeg
!wget -q -nc https://bin.equinox.io/c/4VmDZdvnKvA/ngrok-stable-linux-amd64.zip
import os
if os.path.exists('ngrok-stable-linux-amd64.zip'):
    print("Arquivo zip do ngrok baixado com sucesso.")
    !unzip -qq -n ngrok-stable-linux-amd64.zip
    if os.path.exists('./ngrok'):
        print("ngrok extraído com sucesso.")
    else:
        print("Erro: ngrok não foi extraído corretamente.")
else:
    print("Erro: Arquivo zip do ngrok não foi baixado.")
!pip install pyngrok nest_asyncio -q

In [ ]:
!rm -rf AI-Transcription-FastAPI
!git clone https://github.com/klabacher/AI-Transcription-FastAPI -b Dev

In [ ]:
!pip install -q -r /content/AI-Transcription-FastAPI/requirements.txt
!pip uninstall torch torchvision torchaudio -y -q
!pip install torch torchvision torchaudio -q --index-url https://download.pytorch.org/whl/cu121

In [ ]:
Token = "" # @param {"type":"string"}
import os
os.environ['NGROK_AUTH_TOKEN'] = Token
os.environ['FORCE_CUDA'] = 'True'

In [ ]:
import asyncio
import uvicorn
from pyngrok import ngrok
import os
from google.colab import userdata
import nest_asyncio

!pkill ngrok
!pkill uvicorn

nest_asyncio.apply()
try:

    ngrok_token = os.environ['NGROK_AUTH_TOKEN']
    ngrok.set_auth_token(ngrok_token)
except Exception as e:
    print(f"NGROK Error: {e}")
async def start_server():
    os.chdir('/content/AI-Transcription-FastAPI')
    config = uvicorn.Config("main:app", host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    server_task = asyncio.create_task(server.serve())
    public_url_api = ngrok.connect(8000)
    print("\n\n====================================================================")
    print(f"   API_Link {public_url_api}")
    print("====================================================================")
    await server_task
try:
    asyncio.run(start_server())
except KeyboardInterrupt:
    print("\nServidor interrompido pelo utilizador.")
    ngrok.kill()